In [3]:
import pandas as pd 
import transformers
from transformers import T5Tokenizer

c:\Users\kaczm\.virtualenvs\Abbr_env-FC8IvZ7o\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Let's load the data

train_data_path = 'C:/Users/kaczm/OneDrive/Pulpit/Abbreviation_Disambiguation_PolEval/in_train_a.tsv'
df1 = pd.read_csv(train_data_path, sep='\t', header=None, names=['abbreviation', 'context'])

train_data_path = 'C:/Users/kaczm/OneDrive/Pulpit/Abbreviation_Disambiguation_PolEval/expected_train_a.tsv'
df2 = pd.read_csv(train_data_path, sep='\t', header=None, names=['expanded_abbreviation', 'base_abbreviation'])

In [4]:
# Let's merge the train file 

training_df = pd.merge(df1, df2, left_index=True, right_index=True)

In [5]:
pd.set_option('display.max_colwidth', None)
training_df

,abbreviation,context,expanded_abbreviation,base_abbreviation
0,l.,"ciągników serii 1523 i 1221 otrzymały sześciocylindrowe silniki o pojemności 7,2 <mask> Także te modele nie były wysilone mocowo i osiągały 158 KM w przypadku Belarusa",litra,litr
1,s.,"na zderzaku poprzedzającego auta w odległości zaledwie 1,235 s przy założonych 3 <mask> Przy okazji prawą ręką obsługiwał telefon. Podobnie srebrny Ford S-Max – jego szofer",sekundach,sekunda
2,Sen.,"Kambodża nie wypowiada się na temat wewnętrznych spraw innych państw - powiedział Hun <mask> Agencja Reutera przypomina, że rządzący Kambodżą od ponad 30 lat polityk sam w 1997",Sen.,Sen.
3,s.,"grzechów. Cykl rozpoczęliśmy rozmową z Jerzym Pilchem o pysze; o gniewie mówiła <mask> Małgorzata Chmielewska, o chciwości - Janusz Palikot, o nieumiarkowaniu - Bartłomiej",siostra,siostra
4,b.,"ocenił w rozmowie z PAP poseł Michał Jach. Odniósł się w ten sposób do wypowiedzi <mask> szefa MON Tomasza Siemoniaka, który podczas środowej wizyty w Rzeszowie powiedział:",byłego,były
...,...,...,...,...
4276,s.,"bywa męczący. Poniżej dalsza część artykułu [i]Prószyński i S-ka, Warszawa 2010 <mask> 383[/i] © Licencja na publikację © ℗ Wszystkie prawa zastrzeżone Źródło: Rzeczpospolita",roku,rok
4277,l.,"Lewicy Demokratycznej - 12,5 proc. (l. głosów - 5 950), Konfederacja - 7,76 proc (<mask> głosów - 3 694), Polskie Stronnictwo Ludowe - 4,49 proc. (l. głosów - 2 137). W",strony,strona
4278,bo.,www. wloclawek.pl na stronie dedykowanej włocławskiemu budżetowi obywatelskiemu: <mask> wloclawek.eu oraz na tablicy ogłoszeń w urzędzie miasta. Głosowanie przeprowadzone,i.,i.
4279,a.,"przewodnictwem Prezydenta RP a we Francji nie Zadanie 16. 16.1 funkcja kontrolna 16.2 <mask> politycznej b. premiera c. 231 Zadanie 17. 17. 1 A,E 17.2. C,D Zadanie 18 a. Rada",zeszyt,zeszyt


In [6]:
# Create tokenizer

import sentencepiece
from transformers import MT5Tokenizer

tokenizer = MT5Tokenizer.from_pretrained('google/mt5-large')  


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
# Tokenize data
import torch

def tokenize_data(context, expanded_abbreviation):
    tokenized_input = tokenizer.encode(context, return_tensors='pt', max_length=512, truncation=True)
    tokenized_target = tokenizer.encode(expanded_abbreviation, return_tensors='pt', max_length=512, truncation=True)
    return tokenized_input, tokenized_target

training_df['tokenized_input'], training_df['tokenized_target'] = zip(*training_df.apply(lambda x: tokenize_data(x['context'], x['expanded_abbreviation']), axis=1))


In [9]:
# Save training df
# training_df.to_csv('C:/Users/kaczm/OneDrive/Pulpit/Abbreviation_Disambiguation_PolEval/training_df.csv', index=False)

In [ ]:
from transformers import MT5ForConditionalGeneration

model = MT5ForConditionalGeneration.from_pretrained('google/mt5-large')
# Define your training loop, loss function, optimizer, etc., for fine-tuning
